# AI-Powered Deepfake Detection for Financial Security

**Copyright (c) 2026 Shrikara Kaudambady. All rights reserved.**

This notebook demonstrates a foundational approach to deepfake detection using a Convolutional Neural Network (CNN). We simulate a dataset of 'real' images with natural noise and 'deepfake' images with artificial, periodic noise. The CNN then learns to classify images based on these subtle background artifacts, a common technique in real-world deepfake analysis.

### 1. Setup and Library Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns

### 2. Synthetic Data Generation
We create our own dataset. 'Real' images will have random Gaussian noise, while 'Deepfake' images will have a structured, periodic noise pattern. This mimics the difference between natural camera sensor noise and AI-generated artifacts.

In [ ]:
IMG_SIZE = 64
N_SAMPLES = 2000

def generate_image(is_fake):
    # Create a base gradient image
    base_img = np.zeros((IMG_SIZE, IMG_SIZE))
    x, y = np.meshgrid(np.linspace(-1, 1, IMG_SIZE), np.linspace(-1, 1, IMG_SIZE))
    base_img = (x + y) / 2
    
    if is_fake:
        # Add structured, periodic noise for fakes
        freq = np.random.uniform(5, 15)
        noise = np.sin(x * freq) * np.cos(y * freq) * 0.1
    else:
        # Add random Gaussian noise for real images
        noise = np.random.normal(0, 0.05, (IMG_SIZE, IMG_SIZE))
        
    return np.clip(base_img + noise, -1, 1)

# Generate the dataset
real_images = np.array([generate_image(is_fake=False) for _ in range(N_SAMPLES // 2)])
fake_images = np.array([generate_image(is_fake=True) for _ in range(N_SAMPLES // 2)])

X = np.concatenate([real_images, fake_images])
y = np.array([0] * (N_SAMPLES // 2) + [1] * (N_SAMPLES // 2)) # 0 = real, 1 = fake

# Reshape for the CNN (add a channel dimension)
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print(f"Dataset created. Shape of X: {X.shape}, Shape of y: {y.shape}")

# Visualize an example of each
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(real_images[0], cmap='gray')
axes[0].set_title('Example of a "Real" Image')
axes[1].imshow(fake_images[0], cmap='gray')
axes[1].set_title('Example of a "Deepfake" Image')
plt.show()

### 3. Data Preprocessing and Splitting
We normalize the pixel values and split the data into training and testing sets.

In [ ]:
# Normalize pixel values from [-1, 1] to [0, 1]
X_normalized = (X + 1) / 2.0

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

### 4. Build the CNN Model
We'll create a simple Convolutional Neural Network to learn the different noise patterns.

In [ ]:
model = models.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1)),
    
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid') # Binary classification output
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

### 5. Train and Evaluate the Model

In [ ]:
print("Training the CNN model...")
history = model.fit(
    X_train, y_train,
    epochs=15,
    validation_split=0.2,
    batch_size=32,
    verbose=1
)

# Evaluate on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {accuracy*100:.2f}%")

# Generate predictions
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype('int32')

# Classification Report
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred, target_names=['Real', 'Deepfake']))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Deepfake'], yticklabels=['Real', 'Deepfake'])
plt.title('Confusion Matrix')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()